**Практическая работа #6. Атаки на модели машинного обучения и защита от них**

Работу выполнил студент:

группы: ББМО-02-23

Дурягин Михаил Романович


**Adversarial Robustness Toolbox (ART)** — это инструментарий с открытым исходным кодом, предназначенный для улучшения безопасности машинного обучения. ART предоставляет исследователям и разработчикам инструменты для защиты моделей машинного обучения от враждебных атак, таких как подделка входных данных с целью введения в заблуждение алгоритмов машинного обучения.

Основные возможности ART включают:

* Обнаружение и защита: инструменты для обнаружения и защиты моделей от враждебных атак.
* Атаки: методы для тестирования моделей на устойчивость к враждебным атакам.
* Защитные меры: методы для улучшения устойчивости моделей, включая обучение с защитой от атак.

ART поддерживает широкий спектр фреймворков машинного обучения, таких как TensorFlow и PyTorch, и предлагает модульную архитектуру, позволяющую легко расширять его функциональность.

Для начала установим необходимые библиотеки для работы

In [2]:
!pip install tensorflow
!pip install adversarial-robustness-toolbox

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.4 MB/s eta 0:00:00


Импортируем библиотеки

In [25]:
import tensorflow.compat.v1 as tf
import numpy as np
import matplotlib.pyplot as plt
from art.attacks.evasion import FastGradientMethod
from art.defences.trainer import AdversarialTrainer
from art.estimators.classification import TensorFlowClassifier
from art.utils import load_mnist
tf.compat.v1.disable_eager_execution()  # Добавлено для предотвращения ошибки выполнения Tensorflow.

Для теста, загрузим набор данных MNIST

In [19]:
(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_mnist()

Создадим модель для работы с ней.

In [20]:
input_ph = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
labels_ph = tf.placeholder(tf.int32, shape=[None, 10])

x = tf.layers.conv2d(input_ph, filters=4, kernel_size=5, activation=tf.nn.relu)
x = tf.layers.max_pooling2d(x, 2, 2)
x = tf.layers.conv2d(x, filters=10, kernel_size=5, activation=tf.nn.relu)
x = tf.layers.max_pooling2d(x, 2, 2)
x = tf.layers.flatten(x)
x = tf.layers.dense(x, 100, activation=tf.nn.relu)
logits = tf.layers.dense(x, 10)

loss = tf.reduce_mean(tf.losses.softmax_cross_entropy(logits=logits, onehot_labels=labels_ph))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train = optimizer.minimize(loss)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

<ipython-input-20-d9462c22f1ed>:4: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  x = tf.layers.conv2d(input_ph, filters=4, kernel_size=5, activation=tf.nn.relu)
<ipython-input-20-d9462c22f1ed>:5: UserWarning: `tf.layers.max_pooling2d` is deprecated and will be removed in a future version. Please use `tf.keras.layers.MaxPooling2D` instead.
  x = tf.layers.max_pooling2d(x, 2, 2)
<ipython-input-20-d9462c22f1ed>:6: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  x = tf.layers.conv2d(x, filters=10, kernel_size=5, activation=tf.nn.relu)
<ipython-input-20-d9462c22f1ed>:7: UserWarning: `tf.layers.max_pooling2d` is deprecated and will be removed in a future version. Please use `tf.keras.layers.MaxPooling2D` instead.
  x = tf.layers.max_pooling2d(x, 2, 2)
<ipython-input-20-d9462c22f1ed>:8: UserWarning: `tf.layers.flatten` is

Создаем классификатор ART и тренируем его

In [22]:
classifier = TensorFlowClassifier(
    clip_values=(min_pixel_value, max_pixel_value),
    input_ph=input_ph,
    output=logits,
    labels_ph=labels_ph,
    train=train,
    loss=loss,
    learning=None,
    sess=sess,
    preprocessing_defences=[],
)
classifier.fit(x_train, y_train, batch_size=64, nb_epochs=5)
predictions = classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Точность (тестовая): {}%".format(accuracy * 100))

Точность (тестовая): 97.98%


В качестве примера воспользуемся методом Fast Gradient Sign Method. Он относится к атакам белого ящика, когда у злоумышленника есть доступ к нейросетевой модели и ее коэффициентам. Реализуем и запускаем атаку и выведем точность на тестовых выборках.

In [23]:
attack = FastGradientMethod(estimator=classifier, eps=0.2)
x_test_adv = attack.generate(x=x_test)

predictions = classifier.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Точность (атакованная тестовая): {}%".format(accuracy * 100))

Точность (атакованная тестовая): 56.53%


Как можно заметить, точность на атакованной момели упало.

Далее рассмотрим сценарий защиты предлагаемый ART.

In [33]:
trainer = AdversarialTrainer(classifier, attack)
trainer.fit(x_train, y_train)
predictions_defended = trainer.predict(x_test_adv)
predictions_original = trainer.predict(x_test)

Precompute adv samples:   0%|          | 0/1 [00:00<?, ?it/s]

Adversarial training epochs:   0%|          | 0/20 [00:00<?, ?it/s]